# 1. Wiktionary Parsing

Process a Wiktionary dump to extract synonym relations for a random language (not English, Ukrainian or Russian :). You can find the latest dumps at https://dumps.wikimedia.org/backup-index.html. The task requires application of XML SAX parsing.

In [116]:
from xml.dom.minidom import parse, parseString
import string
import re


DEBUG = False
IN_FILE = "jawiktionary.xml"
OUT_FILE = "jasynonyms.txt"

def parse_dom(in_file):
    dom = parse(in_file)
    pages = dom.getElementsByTagName("page")
    
    return dom


def parse_syn_section(s):
    """Parse wiktionary article content into sections.
    
    :rtype {'sec_name': 'sec_content' ...}"""
    
    patterns = [
        r"[={\[]+syn[}\]=]+\n(.*?)[=]+[{]?",
        r"[={\[]+syn[}\]=]+[：杼:=]?([ \S]+)[\n\r$]+",
        r"[={\[]+syn[}\]=]+[：杼:=]?(.*?)[=$]"
    ]
    
    matches = []
    
    for pattern in patterns:
        m = re.findall(pattern, s, re.M | re.S | re.U)
        matches.extend(m)
    
    if matches:
        return matches
    else:
        return []

    
def parse_synonyms(string_list):
    
    pattern = r"\[\[(\w+)\]\]"
    syns = set([])
    
    for s in string_list:
        m = re.findall(pattern, s)
        for item in m:
            syns.add(item)
    
    return syns


def contains_sec_header(s):
    sec_header_pattern = r'{{(\S+)}}'
    
    res = re.finditer(sec_header_pattern, s)
    
    if len(res) == 1:
        return res[0]
    elif len(res) > 1:
        return [r for r in res]
    elif len(res) < 1:
        return -1


def contains_syn_header(s):
    sec_header_pattern = r'{{syn}}'
    
    if len(re.findall(sec_header_pattern, s)) > 0:
        return True
    else:
        return False
    
def contains_ascii_chars(s, threshold=0):
    """Returns True if string contains more than `threshold` % 
    of ASCII chars (default=0 for no ASCII chars)"""
    
    chars = set(string.ascii_letters)
    n = len(s)
    m = 0
    
    for c in s:
        if c in chars:
            m+=1
    
    if m / n > threshold:
        return True
    else:
        return False


def analyze_synonyms(dom, out_file):

    outfile = open(out_file, "w+")

    art_total = 0
    art_parsed = 0
    art_contains_syn = 0

    for page in pages:
    
        title = page.getElementsByTagName("title")[0].firstChild.data
        try:
            text = page.getElementsByTagName("text")[0].firstChild.data
        except AttributeError:
            if DEBUG:
                print("Error: Article appears to have no text")
        art_total += 1
        
        # For any article that:
        # has no ASCII chars in text (cut off non-Japanese words)
        # has a {{syn}} section
        
        if not contains_ascii_chars(title) and contains_syn_header(text):
            art_contains_syn += 1
        
            syns = parse_synonyms(parse_syn_section(text))
            
            # If article containing {{syn}} is parsed correctly
            if len(syns) > 0:
                art_parsed += 1
                outfile.write(title)
                outfile.write('\n')
                outfile.write(str(syns))
                outfile.write('\n\n')
                
            # If the article has a section header but nothing is parsed
            elif len(syns) == 0:
                if DEBUG:
                    print(title)
                    print(text)
                    
        # Limiter for debug            
        if DEBUG and art_total > 10000:
            break

    print("Parsed: {0}/{1} – {2}%".format(art_parsed, art_total, art_parsed / art_total * 100))
    print("Contains {{syn}}: {0} or {1}%".format(art_contains_syn, art_contains_syn / art_total * 100))

# dom = parse_dom(IN_FILE)
analyze_synonyms(dom, OUT_FILE)
    

Parsed: 5421/244834 – 2.2141532630271943%
Contains {syn}: 9408 or 3.8426035599630772%


## Спостереження

Для парсингу XML ми надали перевагу моделі `minidom` над моделлю `SAX`, тому що перша виявилася набагато мінімалістичнішою та простішою у розгортанні за реалізацію `SAX` у мові `Python`. При цьому модель `minidom` також задовільно впоралася із отриманням тексту з `XML`-файлу.

Щодо ефективності парсера тексту всередині статей, наш пасер покриває приблизно 2% всієї кількості статей у японському Wiktionary, що стосуються власне японських слів. При цьому:

#### Всього статей: 244834

#### Без відсікання не-японського тексту:

Кількість із заголовком {{syn}}: 19002 або 7.76% від усіх статей

Розпізнано парсером: 10271 або 4.195%

#### З відсіканням латиничного тексту:

Кількість із заголовком {{syn}}: 9408 або 3.84%

Розпізнано парсером: 5421 або 2.21%

## Висновки

* У японському Wiktionary приблизно половина статей є допоміжними або такими, що не стосуються власне японських слів. У більш розвиненому парсері японські слова можна було б розрізняти за ознакою наявності у статті плашки `[[category:{{jpn}}]]`, але для швидкості ми обмежилися простим тестом на наявність латиничних символів;
* Парсер покриває трохи більше половини всіх статей, що мають розділ `{{syn}}`. Відносно низький відсоток розпізнавання парсера обумовлений тим, що заголовки і вміст розділу із синонімами оформлений неоднорідно. Через це досить важко знайти універсальній набір фільтрів (рег.виразів), який би покривав усі можливі варіанти оформлення з однієї сторони, і не забирав би помилкво текст із неспоріднених розділів з іншої сторони. Позаяк парсер можна надалі покращувати за рахунок додавання нових і кормгування існуючих регулярних виразів.


